# Step 0.1. Import necessary libraries 

In [1]:
# Standard python libraries
import logging
import os
import time
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.base import AutoML
from lightautoml.automl.blend import WeightedBlender
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.linear_sklearn import LinearLBFGS
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml.pipelines.features.linear_pipeline import LinearFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task
from lightautoml.utils.profiler import Profiler
from lightautoml.utils.timer import PipelineTimer

# Step 0.2. Parameters 

In [2]:
N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 600 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name

# Step 0.3. Fix torch number of threads and numpy seed 

In [3]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

# Step 0.4. Change profiling decorators settings 

By default, profiling decorators are turned off for speed and memory reduction. If you want to see profiling report after using LAMA, you need to turn on the decorators using command below: 

In [4]:
p = Profiler()
p.change_deco_settings({'enabled': True})

ALL_FUNCS len = 418


# Step 0.5. Example data load 

In [5]:
%%time

data = pd.read_csv('./example_data/test_data_files/sampled_app_train.csv')
data.head()

CPU times: user 76.5 ms, sys: 4.37 ms, total: 80.9 ms
Wall time: 80.2 ms


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,313802,0,Cash loans,M,N,Y,0,270000.0,327024.0,15372.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,319656,0,Cash loans,F,N,N,0,108000.0,675000.0,19737.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,207678,0,Revolving loans,F,Y,Y,2,112500.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
3,381593,0,Cash loans,F,N,N,1,67500.0,142200.0,9630.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
4,258153,0,Cash loans,F,Y,Y,0,337500.0,1483231.5,46570.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0


# Step 0.6. (Optional) Some user feature preparation 

Cell below shows some user feature preparations to create task more difficult (this block can be omitted if you don't want to change the initial data):

In [6]:
%%time

data['BIRTH_DATE'] = (np.datetime64('2018-01-01') + data['DAYS_BIRTH'].astype(np.dtype('timedelta64[D]'))).astype(str)
data['EMP_DATE'] = (np.datetime64('2018-01-01') + np.clip(data['DAYS_EMPLOYED'], None, 0).astype(np.dtype('timedelta64[D]'))
                    ).astype(str)

data['constant'] = 1
data['allnan'] = np.nan

data['report_dt'] = np.datetime64('2018-01-01')

data.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis=1, inplace=True)

[2020-11-11 22:41:27,135] (INFO): Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[2020-11-11 22:41:27,135] (INFO): Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[2020-11-11 22:41:27,136] (INFO): NumExpr defaulting to 8 threads.


CPU times: user 110 ms, sys: 5.99 ms, total: 116 ms
Wall time: 113 ms


# Step 0.7. Create fake multiclass target 

In [7]:
data[TARGET_NAME] = np.where(np.random.rand(data.shape[0]) > .5, 2, data[TARGET_NAME].values)
data[TARGET_NAME].value_counts()

2    4924
0    4677
1     399
Name: TARGET, dtype: int64

# Step 0.8. (Optional) Data splitting for train-test 

Block below can be omitted if you are going to train model only or you have specific train and test files:

In [8]:
%%time

train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE, 
                                         stratify=data[TARGET_NAME], 
                                         random_state=RANDOM_STATE)
logging.info('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

[2020-11-11 22:41:27,233] (INFO): Data splitted. Parts sizes: train_data = (8000, 125), test_data = (2000, 125)


CPU times: user 13.9 ms, sys: 231 µs, total: 14.1 ms
Wall time: 13.4 ms


In [9]:
train_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BIRTH_DATE,EMP_DATE,constant,allnan,report_dt
3273,293399,2,Revolving loans,F,Y,N,0,292500.0,765000.0,38250.0,...,0.0,0.0,0.0,0.0,0.0,1980-08-03,2013-06-26,1,NaN,2018-01-01
812,426509,1,Cash loans,F,N,Y,1,225000.0,349258.5,17959.5,...,0.0,0.0,0.0,0.0,5.0,1984-05-06,2013-02-17,1,NaN,2018-01-01
9630,403114,0,Cash loans,M,Y,Y,2,135000.0,922500.0,29889.0,...,0.0,0.0,4.0,0.0,4.0,1984-01-23,2009-06-01,1,NaN,2018-01-01
7757,366682,2,Cash loans,F,N,N,0,254475.0,1485000.0,39303.0,...,0.0,0.0,0.0,0.0,0.0,1971-11-28,2016-01-10,1,NaN,2018-01-01
9973,289050,0,Cash loans,F,N,Y,0,112500.0,260640.0,25906.5,...,NaN,NaN,NaN,NaN,NaN,1966-11-27,2002-07-12,1,NaN,2018-01-01


# ========= AutoML creation =========

![AutoML pipeline for this task](imgs/tutorial_2_pipeline.png)


## Step 1. Create Timer for pipeline

Here we are going to use strict timer for AutoML pipeline, which helps not to go outside the limit:

In [10]:
%%time

timer = PipelineTimer(600, mode=2)

CPU times: user 54 µs, sys: 93 µs, total: 147 µs
Wall time: 149 µs


## Step 2. Create feature selector

In [11]:
%%time

timer_gbm = timer.get_task_timer('gbm') # Get task timer from pipeline timer 
feat_sel_0 = LGBSimpleFeatures()
mod_sel_0 = BoostLGBM(timer=timer_gbm)
imp_sel_0 = ModelBasedImportanceEstimator()
selector_0 = ImportanceCutoffSelector(feat_sel_0, mod_sel_0, imp_sel_0, cutoff=0, )

CPU times: user 1.36 ms, sys: 0 ns, total: 1.36 ms
Wall time: 1.17 ms


/mnt/alex/LightAutoML/lightautoml/utils/timer.py:210: UserWarning:

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer



## Step 3.1. Create GBMs pipeline for AutoML 

Our GBMs ML pipeline:
- Advanced features for gradient boosting built on selected features (using step 2) 
- 2 different models:
    * LightGBM with params tuning (using OptunaTuner)
    * LightGBM with heuristic params


In [12]:
%%time 

feats_gbm_0 = LGBAdvancedPipeline(top_intersections=4, 
                                  output_categories=True, 
                                  feats_imp=imp_sel_0)
timer_gbm_0 = timer.get_task_timer('gbm')
timer_gbm_1 = timer.get_task_timer('gbm')

gbm_0 = BoostLGBM(timer=timer_gbm_0)
gbm_1 = BoostLGBM(timer=timer_gbm_1)

tuner_0 = OptunaTuner(n_trials=20, timeout=30, fit_on_holdout=True)
gbm_lvl0 = MLPipeline([
        (gbm_0, tuner_0),
        gbm_1
    ],
    pre_selection=selector_0,
    features_pipeline=feats_gbm_0, 
    post_selection=None
)

CPU times: user 725 µs, sys: 0 ns, total: 725 µs
Wall time: 727 µs


## Step 3.2. Create linear pipeline for AutoML 

Our linear pipeline:
- Using features, special for linear models
- LinearLBFGS as a model
- Without feature selection here

In [13]:
%%time

feats_reg_0 = LinearFeatures(output_categories=True, 
                             sparse_ohe='auto')

timer_reg = timer.get_task_timer('reg')
reg_0 = LinearLBFGS(timer=timer_reg)

reg_lvl0 = MLPipeline([
        reg_0
    ],
    pre_selection=None,
    features_pipeline=feats_reg_0, 
    post_selection=None
)

CPU times: user 199 µs, sys: 341 µs, total: 540 µs
Wall time: 542 µs


## Step 4. Create multiclass task and reader

In [14]:
%%time 

task = Task('multiclass', metric = 'crossentropy', ) 
reader = PandasToPandasReader(task = task, samples = None, max_nan_rate = 1, max_constant_rate = 1,
                              advanced_roles = True, drop_score_co = -1, n_jobs = 4)

CPU times: user 3.38 ms, sys: 413 µs, total: 3.8 ms
Wall time: 3.42 ms


## Step 5. Create blender for 2nd level 

To combine predictions from different models into one vector we use WeightedBlender:

In [15]:
%%time

blender = WeightedBlender()

CPU times: user 33 µs, sys: 57 µs, total: 90 µs
Wall time: 92.5 µs


## Step 6. Create AutoML pipeline

In [16]:
%%time

automl = AutoML(reader=reader, levels=[
    [gbm_lvl0, reg_lvl0]
], timer=timer, blender=blender, skip_conn=False)

CPU times: user 143 µs, sys: 244 µs, total: 387 µs
Wall time: 390 µs


## Step 7. Train AutoML on loaded data 

In cell below we train AutoML with target column `TARGET` to receive fitted model and OOF predictions:

In [17]:
%%time

oof_pred = automl.fit_predict(train_data, roles={'target': TARGET_NAME})
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

/mnt/alex/LightAutoML/lightautoml/reader/guess_roles.py:351: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Feats was rejected during automatic roles guess: []
Train process start. Time left 595.8033359050751 secs
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.840073
Early stopping, best iteration is:
[35]	valid's multi_logloss: 0.82727
Time history [3.515303611755371]. Time left 130.33707398176193


/mnt/alex/LightAutoML/lightautoml/utils/timer.py:210: UserWarning:

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer

[I 2020-11-11 22:41:36,491] A new study created in memory with name: no-name-29321ca5-b62a-41fc-a4d5-4897a4d70a11


Optuna may run 159.39274446169534 secs
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.824606


[I 2020-11-11 22:41:40,189] Trial 0 finished with value: -0.8211659893766046 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 108}. Best is trial 0 with value: -0.8211659893766046.


...

[100]	valid's multi_logloss: 0.826275


[I 2020-11-11 22:42:07,560] Trial 6 finished with value: -0.8167450121231377 and parameters: {'feature_fraction': 0.8540362888980227, 'num_leaves': 165}. Best is trial 6 with value: -0.8167450121231377.


Early stopping, best iteration is:
[58]	valid's multi_logloss: 0.816745
Time history [5.529229402542114]. Time left 8999999994.46891
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.826275
Early stopping, best iteration is:
[58]	valid's multi_logloss: 0.816745
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.835202
Early stopping, best iteration is:
[37]	valid's multi_logloss: 0.823403
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.847846
Early stopping, best iteration is:
[19]	valid's multi_logloss: 0.827287
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.844072
Early stopping, best iteration is:
[16]	valid's multi_logloss: 0.826379
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.841693
Early stopping, best iteration is:
[9]	valid's multi_logloss: 0.829459
Time his

[2020-11-11 22:42:58,219] (INFO): oof_pred:
array([[0.45769578, 0.02285249, 0.5194518 ],
       [0.41589692, 0.06399862, 0.52010447],
       [0.4807197 , 0.04082523, 0.47845513],
       ...,
       [0.40053958, 0.02561985, 0.5738406 ],
       [0.48938653, 0.01566021, 0.49495333],
       [0.41111797, 0.02727515, 0.5616069 ]], dtype=float32)
Shape = (8000, 3)


Blending, iter 2: score = -0.8198816331066191, weights = [0.28605267 0.2077377  0.5062097 ]
Blending, iter 3: score = -0.8198816342875361, weights = [0.28632912 0.20697202 0.5066989 ]
Blending, iter 4: score = -0.819881631154567, weights = [0.28690892 0.20681122 0.5062799 ]
CPU times: user 11min 2s, sys: 6.2 s, total: 11min 8s
Wall time: 1min 30s


## Step 8. Predict to test data and check scores

In [18]:
%%time

test_pred = automl.predict(test_data)
logging.debug('Prediction for test data:\n{}\nShape = {}'
              .format(test_pred, test_pred.shape))

logging.info('Check scores...')
logging.info('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))
logging.info('TEST score: {}'.format(log_loss(test_data[TARGET_NAME].values, test_pred.data)))

[2020-11-11 22:42:58,685] (INFO): Check scores...
[2020-11-11 22:42:58,688] (INFO): OOF score: 0.819881631154567
[2020-11-11 22:42:58,690] (INFO): TEST score: 0.8231379891037941


CPU times: user 1.05 s, sys: 23.2 ms, total: 1.08 s
Wall time: 466 ms


## Step 9. Check AUCs for each class in train and test data 

In [19]:
for dat, df, name in zip([oof_pred, test_pred], [train_data, test_data], ['train', 'test']):
    logging.debug('Check aucs {0}...'.format(name))
    for cl in range(3):
        sc = roc_auc_score((df[TARGET_NAME].values == cl).astype(np.float32), dat.data[:, cl])
        logging.info('Class {0} {1} auc score: {2}'.format(cl, name, sc))

[2020-11-11 22:42:58,698] (INFO): Class 0 train auc score: 0.5522930835508424
[2020-11-11 22:42:58,701] (INFO): Class 1 train auc score: 0.7219001901447165
[2020-11-11 22:42:58,704] (INFO): Class 2 train auc score: 0.5433913724560568
[2020-11-11 22:42:58,706] (INFO): Class 0 test auc score: 0.5395174612738822
[2020-11-11 22:42:58,708] (INFO): Class 1 test auc score: 0.7340169270833333
[2020-11-11 22:42:58,710] (INFO): Class 2 test auc score: 0.5207616713760597


## Step 10. Profiling AutoML 

To build report here, we must turn on decorators on step 0.4. Report is interactive and you can go as deep into functions call stack as you want:

In [20]:
%%time
p.profile('my_report_profile.html')
assert os.path.exists('my_report_profile.html'), 'Profile report failed to build'

FULL_STATS_DF shape = (9797, 6)
RUN_FNAME vc head:
NumpyDataset.set_data [28]         1
WeightedBlender._get_scorer [8]    1
NumpyDataset._check_dtype [92]     1
CatLinear.forward [759]            1
PandasDataset._get_rows [33]       1
Name: run_fname, dtype: int64
CONNECTED COMPONENTS cnt = 1
PATH LENS describe:
count    9798.000000
mean        8.022147
std         2.714216
min         0.000000
25%         6.000000
50%         8.000000
75%         9.000000
max        20.000000
dtype: float64
CPU times: user 1.03 s, sys: 20 ms, total: 1.05 s
Wall time: 1.04 s


# Appendix. Profiling report screenshots 

After loading HTML with profiling report, you can see fully folded report (please wait for green LOAD OK text for full load finish). If you click on triangle on the left, it unfolds and look like this:  

<img src="imgs/tutorial_3_initial_report.png" alt="Initial profiling report" style="width: 500px;"/>

If we go even deeper we will receive situation like this:

<img src="imgs/tutorial_3_unfolded_report.png" alt="Profiling report after several unfoldings on different levels" style="width: 500px;"/>
